In [1]:
!pip install ultralytics gradio  # Install dependencie

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.4/911.4 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:


import os
import cv2
import torch
import gradio as gr
from ultralytics import YOLO
import torch.nn as nn

# Define CNN-LSTM Model
class CNNLSTMModel(nn.Module):
    def __init__(self):
        super(CNNLSTMModel, self).__init__()
        # CNN layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        # LSTM layer
        self.lstm = nn.LSTM(input_size=64 * 32 * 32, hidden_size=128, batch_first=True)
        # Fully connected layers
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 2)  # 2 classes: Fall and Non-Fall

    def forward(self, x):
        # CNN layers
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        # Flatten
        x = x.view(x.size(0), -1)
        # LSTM layer
        x, (hn, cn) = self.lstm(x.unsqueeze(1))  # Add sequence dimension
        # Fully connected layers
        x = torch.relu(self.fc1(hn[-1]))  # Use the last hidden state
        x = self.fc2(x)
        return x

# Load YOLO model
yolo_model_path = '/content/yolov5su.pt'  # Replace with your YOLO model path
yolo_model = YOLO(yolo_model_path)

# Load CNN-LSTM model
cnn_lstm_model_path = '/content/CNN_LSTM (2).pth'  # Replace with your model path
cnn_lstm_model = CNNLSTMModel()
cnn_lstm_model.load_state_dict(torch.load(cnn_lstm_model_path, map_location=torch.device('cpu')))
cnn_lstm_model.eval()

# Preprocess function for frames
def preprocess_frame(frame, target_size=(128, 128)):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    resized_frame = cv2.resize(frame, target_size)  # Resize to target size
    normalized_frame = resized_frame / 255.0  # Normalize
    return torch.tensor(normalized_frame).unsqueeze(0).unsqueeze(0).float()  # Convert to tensor

# Define video processing function
def process_video(input_video):
    cap = cv2.VideoCapture(input_video)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    output_video_path = "output_video.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # YOLO object detection
        results = yolo_model(frame)
        results = results[0]

        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())

            if cls == 0 and conf > 0.5:  # Process 'person' class only
                aspect_ratio = (y2 - y1) / (x2 - x1)
                label = 'Fall' if aspect_ratio < 1.5 else 'Non-Fall'
                color = (0, 0, 255) if label == 'Fall' else (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Write annotated frame
        out.write(frame)

    cap.release()
    out.release()
    return output_video_path

# Gradio Interface
def gradio_video(input_video):
    output_video = process_video(input_video)
    return output_video

# Gradio Interface with Enhanced Styling
gr.Interface(
    fn=gradio_video,
    inputs=gr.Video(label="🎥 Upload a Video"),  # Add an emoji for style
    outputs=gr.Video(label="📹 Annotated Output"),
    title="<h1 style='color: #F8C471; text-shadow: 2px 2px 4px #000000;'>🌟 Fall Detection System 🌟</h1>",
    description=(
        "<div style='background-color: #1A5276; padding: 20px; border-radius: 15px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.3);'>"
        "<h3 style='color: #F7DC6F; font-weight: bold; text-align: center;'>🚨 Detect Falls Using YOLO and CNN-LSTM Models</h3>"
        "<p style='color: #D6EAF8; font-size: 1.1em;'>"
        "Upload a video to analyze and detect falls. Detected individuals are highlighted with "
        "<span style='color: #FF5733;'>Fall</span> or "
        "<span style='color: #58D68D;'>Non-Fall</span> labels using bounding boxes.</p>"
        "</div>"
    ),
    theme="default",  # Optional; customize if needed
    css=(
        "body {background: linear-gradient(to right, #2980B9, #6DD5FA); color: #ECF0F1; font-family: 'Arial', sans-serif;} "
        ".gradio-interface {border-radius: 20px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2); font-family: 'Roboto', sans-serif;} "
        "h1 {text-align: center; font-size: 2.8em; margin-top: 20px;} "
        "h3 {font-size: 1.7em; text-align: center; font-weight: bold; padding-bottom: 15px;} "
        "p {text-align: center; font-size: 1.2em; line-height: 1.6;} "
        ".gradio-video {border-radius: 15px; border: 3px solid #2980B9;} "
        ".gradio-button {background-color: #F39C12; color: #2C3E50; font-weight: bold; border-radius: 10px;} "
        ".gradio-button:hover {background-color: #F1C40F;}"
    )
).launch()




Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 17.7M/17.7M [00:00<00:00, 106MB/s] 


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1124a409b3377a015f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
